1. 安装pytorch geometric

In [ ]:
# !pip uninstall torch torch-scatter torch-sparse torch-spline-conv torch-geometric
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110  torchaudio===0.7.2 torchtext==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.1+cu110.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.1+cu110.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.1+cu110.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.1+cu110.html
!pip install torch-geometric 
!pip install tensorboardX
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

2. 导入需要的包

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime
import math

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid

# 增加实验所使用的数据集
from torch_geometric.datasets import Flickr
from torch_geometric.datasets import Reddit2

from torch_geometric.data import DataLoader

import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

3. 自定义GAT层

In [33]:
class GATSelf(pyg_nn.MessagePassing):
    def __init__(self,in_features,out_featrues):
        super(GATSelf,self).__init__(aggr='add')  # aggregation function
        self.a = nn.Parameter(torch.zeros(size=(2*out_featrues, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)  # 初始化
        self.linear=nn.Linear(in_features,out_featrues)
    def forward(self,x,edge_index):
        x=self.linear(x)
        N=x.size()[0]
        row, col=edge_index
        a_input = torch.cat([x[row], x[col]], dim=1)
        print('a_input.size',a_input.size())
        # [N, N, 1] => [N, N] 图注意力的相关系数（未归一化）
        temp=torch.mm(a_input,self.a).squeeze()
        print('temp.size',temp.size())
        e = F.leaky_relu(temp)
        print('e',e)
        print('e.size', e.size())
        #e_all为同一个节点与其全部邻居的计算的分数的和，用于计算归一化softmax
        e_all=torch.zeros(x.size()[0])
        count = 0
        for i in col:
            e_all[i]+=e[count]
            count=count+1
        print('e_all',e_all)

        for i in range(len(e)):
            e[i]=math.exp(e[i])/math.exp(e_all[col[i]])
        print('attention',e)
        print('attention.size',e.size())

        return self.propagate(edge_index,x=x,norm=e)

    def message(self, x_j, norm):
        print('x_j:', x_j)
        print('x_j.size', x_j.size())
        print('norm', norm)
        print('norm.size', norm.size())
        print('norm.view.size', norm.view(-1, 1).size())
        return norm.view(-1, 1) * x_j


    def update(self, aggr_out):
         return aggr_out

4. 组装GAT

In [34]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.gat = GATSelf(dataset.num_node_features,16)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x, training=self.training)
        x = self.gat(x, edge_index)
        print('x_gat',x.size())
        return F.log_softmax(x, dim=1)

5.训练与测试

In [ ]:
dataset = Planetoid(root='Cora', name='Cora')

model=Net()
data=dataset[0]
out=Net()(data)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct/int(data.test_mask.sum())
print('Accuracy:{:.4f}'.format(acc))